In [1]:

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

df=pd.read_csv('KeyWords2(wjx).csv',header=None)
df=df.drop([0],axis=1)   #删除0、1列（都是序号）
df=df.drop([1],axis=0)
df.columns=range(df.shape[1])     
display(df)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,name,genre,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
2,3rd Revelation,Black Metal,2,1,1,1,2,1,0,2,...,1,0,2,3,3,0,1,3,0,1
3,5 Days Bleeding,Melodic Death/Power/Black Metal,0,0,0,2,0,0,2,4,...,0,1,1,2,0,0,2,5,2,2
4,5 Days Bleeding,Melodic Death/Power/Black Metal,0,0,0,0,0,0,1,1,...,1,0,0,0,2,1,3,3,1,1
5,5 Days Bleeding,Melodic Death/Power/Black Metal,1,0,0,0,1,0,0,1,...,0,1,0,2,0,2,1,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238848,暴君,Melodic Black/Folk Metal (early); Melodic Deat...,1,0,0,1,1,0,1,2,...,0,0,2,0,0,0,2,0,0,0
238849,暴君,Melodic Black/Folk Metal (early); Melodic Deat...,0,0,0,0,0,0,1,1,...,0,0,1,2,0,0,0,0,0,1
238850,暴君,Melodic Black/Folk Metal (early); Melodic Deat...,0,2,0,1,0,2,2,4,...,0,0,4,1,1,1,2,0,0,1
238851,暴君,Melodic Black/Folk Metal (early); Melodic Deat...,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,2,0,1,0


In [2]:
# train和test的分界位置：前2/3用作训练，后1/3用作测试
M=np.int32(df.shape[0]*2/3)
IVarCnt=df.shape[1]-2          #自变量个数: 注意最后两列是歌曲名称、歌曲类型
print(M,'/',df.shape[0],'\n')

159234 / 238852 



In [3]:
# 自变量
ColSet=range(2,IVarCnt+2)                 #自变量在数据集中的位置（列）
x_train=df.iloc[2:M,ColSet].values   
x_test= df.iloc[M:, ColSet].values   

print('\nx_train',x_train.shape)
print(x_train)
print('\nx_test',x_test.shape)
print(x_test)


x_train (159232, 23)
[[0 0 0 ... 5 2 2]
 [0 0 0 ... 3 1 1]
 [1 0 0 ... 2 1 0]
 ...
 [0 0 0 ... 1 2 1]
 [0 0 0 ... 2 1 3]
 [2 0 0 ... 2 1 0]]

x_test (79618, 23)
[[0 0 0 ... 1 0 1]
 [0 0 0 ... 3 2 0]
 [0 1 1 ... 3 1 0]
 ...
 [0 2 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 2 0 0]]


In [4]:
# 为因变量准备原始数据
Y=df.iloc[:,1].values     #第1列，即：歌曲类型
Y_train=Y[2:M]
Y_test= Y[M:]

print('\nY_train',Y_train.shape)
print(Y_train)
print('\nY_test',Y_test.shape)
print(Y_test)

print('\nY',Y.shape)
print(Y)


Y_train (159232,)
['Melodic Death/Power/Black Metal' 'Melodic Death/Power/Black Metal'
 'Melodic Death/Power/Black Metal' ... 'Death/Thrash Metal'
 'Death/Thrash Metal' 'Death/Thrash Metal']

Y_test (79618,)
['Death/Thrash Metal' 'Death/Thrash Metal' 'Death/Thrash Metal' ...
 'Melodic Black/Folk Metal (early); Melodic Death/Folk Metal (later)'
 'Melodic Black/Folk Metal (early); Melodic Death/Folk Metal (later)'
 'Melodic Black/Folk Metal (early); Melodic Death/Folk Metal (later)']

Y (238852,)
['genre' 'Black Metal' 'Melodic Death/Power/Black Metal' ...
 'Melodic Black/Folk Metal (early); Melodic Death/Folk Metal (later)'
 'Melodic Black/Folk Metal (early); Melodic Death/Folk Metal (later)'
 'Melodic Black/Folk Metal (early); Melodic Death/Folk Metal (later)']


In [5]:
def IsBandType(song,typeidx):
    s=song
    Arr=[]
    while True:
        p=s.find('/')
        if(p>0):
            Arr.append(s[0:p])
            s=s[p+1:]
        else:
            Arr.append(s)
            break
    b=False
    for s in Arr:
        for type in SongType[typeidx]:
            if s==type:
                b=True
                break
        if b:
            break       
    return b



 
# 回归模型预测值   


def activation(X,w,b):
    return (np.dot(X,w)+b)      # dot(X,w): 两个向量的点乘

def loss(X,y, w, b):
    dif=y- activation(X, w,b)
    return np.sum(dif**2/(2*len(y)),keepdims=True)

def accuracy(X, y, w, b):
    pre = predict(X, w, b)
    return np.sum(np.where(pre==y,1,0))/len(y)    
 
def predict(X, w, b):
    result = np.int32(activation(X,w,b)+0.5)    #                      
    return result

def update(X, y, w, b, eta):  #将w和b的值分别稍微调大，根据损失函数的变化，调整w和b
    h =1e-4
    loss_origin = loss(X, y, w, b)                       # 损失函数
    delta_w=np.zeros_like(w) 
    delta_b=np.zeros_like(b) 
    
    for i in range(len(w)):
        tmp=w[i]
        w[i]+=h                                          # 将参数w中的一个值稍微调大一点 
        loss_after=loss(X,y,w,b)                         # 损失函数
        delta_w[i] =eta*(loss_after - loss_origin)/h     # 损失函数的增量
        w[i]=tmp 
        
    tmp=b 
    b += h                                                # 将参数b中的一个值稍微调大一点 
    loss_after = loss(X, y, w, b)
    delta_b=eta*(loss_after-loss_origin)/h
    b=tmp
        
    w-=delta_w    # 更新参数值 
    b-=delta_b 
    
    #print('Delta:',delta_w,delta_b)
    return w,b

In [7]:
IsBandType('Melodic Death/Power/Black Metal',0)

True

In [8]:
def my_logistic():
    from sklearn.linear_model import LogisticRegression
    model=LogisticRegression()
    model.fit(x_train,y_train)
    
    weights=model.coef_    
    bias=model.intercept_
    y_pred_train = model.predict(x_train)
    r_sq_train = model.score(x_train, y_train)
    
    y_pred_test = model.predict(x_test)
    r_sq_test = model.score(x_test, y_test)
    
    print('weights:',weights)
    print('bias:',bias)
    print('R2 train:',r_sq_train)
    print('R2 test:',r_sq_test)

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

for m in range(len(SongType)):
    print('\n-------------------',m,SongType[m][0],'--------------------\n')
    
    # 将Y中的歌曲类型转化为数值0-1: 根据是否为第m个歌曲类型，
    # 然后把转化后的y作为因变量
    y=np.zeros(df.shape[0]) # 用于转化为数值0-1，代表是否是第m类歌曲 

    for i in range(len(Y)):
        if IsBandType(Y[i],m):
            y[i]=1     # 转化为数值0-1
        else:
            y[i]=0
    print('y=',y)
    print('count=',np.sum(y)) 

    # 确定训练用的因变量、测试用的因变量
    y_train=y[2:M]                          
    y_test= y[M:]
    
    
    print('x_train:',x_train)
    print('length:',len(x_train))
    
    print('x_test:',x_test)
    print('length:',len(x_test))
    
    print('y_train:',y_train)
    print('length:',len(y_train))
    
    print('y_test:',y_test)
    print('length:',len(y_test))
    
    # 初试化w和b，都取0.1
    weights=np.ones(IVarCnt)/10 
    bias=np.ones(1)/10
    
    ##建立模型：
    
    
    for k in range(20): # 让模型进行多次学习 
        weights, bias = update(x_train ,y_train, weights, bias,eta=0.01)
    
    pre=predict(x_test, weights, bias)
    acc=accuracy(x_test, y_test, weights, bias)
    
    ####r_sq = model.score(x_test, y_test)
    #weights=model.coef_    
    #bias=model.intercept_
    
    
    #predict(x_train, weights, bias)
    #print('my prediction:',predict(x_train, weights, bias))
    #print('my accuracy:',accuracy(x_train, y_train, weights, bias))
    
    
    
     # 打印模型准确度
    print('Accuracy=%.2f%%' % (100*acc))
        
        
    # 打印模型的系数  
    print('w=',weights)
    print('b=',bias)
    
    pre=predict(x_test,weights,bias)                # 预测值（歌曲是否第m类）
    if m==0:      
        pre=pre.reshape(-1,1)
        PreArr=pre                                  # 保存预测值（歌曲是否第m类）
        y_test=y_test.reshape(-1,1)
        ActArr=y_test                               # 保存实际值（歌曲是否第m类） 
    else:    
        PreArr=np.insert(PreArr,m,pre,axis=1)       # 保存预测值（歌曲是否第m类）  
        ActArr=np.insert(ActArr,m,y_test,axis=1)    # 保存实际值（歌曲是否第m类）




------------------- 0 Black Metal --------------------

y= [0. 1. 1. ... 0. 0. 0.]
count= 57688.0
x_train: [[0 0 0 ... 5 2 2]
 [0 0 0 ... 3 1 1]
 [1 0 0 ... 2 1 0]
 ...
 [0 0 0 ... 1 2 1]
 [0 0 0 ... 2 1 3]
 [2 0 0 ... 2 1 0]]
length: 159232
x_test: [[0 0 0 ... 1 0 1]
 [0 0 0 ... 3 2 0]
 [0 1 1 ... 3 1 0]
 ...
 [0 2 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 2 0 0]]
length: 79618
y_train: [1. 1. 1. ... 0. 0. 0.]
length: 159232
y_test: [0. 0. 0. ... 0. 0. 0.]
length: 79618
Accuracy=74.09%
w= [ 0.02961889  0.0558543   0.06788425  0.03428142  0.02961889  0.0558543
  0.04271364 -0.02757376  0.04271535  0.01672668  0.02961889  0.0660304
 -0.03774402  0.05076262  0.02158953  0.0143992  -0.00260775  0.01333691
  0.04834118 -0.00482755 -0.0487582   0.04237799  0.02740336]
b= [0.03853095]

------------------- 1 Black --------------------

y= [0. 0. 0. ... 0. 0. 0.]
count= 19918.0
x_train: [[0 0 0 ... 5 2 2]
 [0 0 0 ... 3 1 1]
 [1 0 0 ... 2 1 0]
 ...
 [0 0 0 ... 1 2 1]
 [0 0 0 ... 2 1 3]
 [2 0

Accuracy=99.56%
w= [ 0.02444799  0.05304171  0.06280801  0.0266872   0.02444799  0.05304171
  0.03720192 -0.04205568  0.03658109  0.00691051  0.02444799  0.06139122
 -0.05532381  0.04171821  0.01077465  0.00259669 -0.01216161  0.00691715
  0.0446833  -0.01103253 -0.0635947   0.03613258  0.01918095]
b= [0.02308973]

------------------- 11 Blackened Melodic Death Metal --------------------

y= [0. 0. 0. ... 1. 1. 1.]
count= 7171.0
x_train: [[0 0 0 ... 5 2 2]
 [0 0 0 ... 3 1 1]
 [1 0 0 ... 2 1 0]
 ...
 [0 0 0 ... 1 2 1]
 [0 0 0 ... 2 1 3]
 [2 0 0 ... 2 1 0]]
length: 159232
x_test: [[0 0 0 ... 1 0 1]
 [0 0 0 ... 3 2 0]
 [0 1 1 ... 3 1 0]
 ...
 [0 2 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 2 0 0]]
length: 79618
y_train: [0. 0. 0. ... 0. 0. 0.]
length: 159232
y_test: [0. 0. 0. ... 1. 1. 1.]
length: 79618
Accuracy=96.14%
w= [ 0.02486452  0.05340643  0.0634435   0.02723639  0.02486452  0.05340643
  0.03782915 -0.04032529  0.03723143  0.00808559  0.02486452  0.06209246
 -0.05362272  0.042543

In [6]:
T00=['Black Metal']
T01=['Black']
T02=['Atmospheric Black Metal']
T03=['Melodic Black Metal']
T04=['Experimental Black',  'Blackened Deathcore',  'Blackened Epic Doom',  
    'Black \'n\' Roll',  'Black Metal, Neofolk',  'Raw Black Metal (early)',  
    'Experimental Post-Black Metal',  'Blackened Death',  'Psychedelic Black Metal, Punk',  
    'Progressive Black Metal',  'Blackened Industrial',  'Post-Black'] 
T05=['Raw Black Metal, Ambient',  'Melodic Blackened Death Metal',  'Blackened Death Metal (early)',  
    'Viking Black Metal',  'Melodic Post-Black',  'Black Metal, Punk',  
    'Blackened Symphonic Metal',  'Blackened Doom Metal',  'Black Thrash',  
    'Psychedelic Black',  'Blackened Sludge',  'Black Metal, Ambient'] 
T06=['Ambient Black',  'Avant-garde Black',  'Depressive Black Metal, Ambient',  
    'Blackened Funeral Doom Metal',  'Blackened Thrash',  'Epic Folk Black',  
    'Industrial Black',  'Experimental Post-Black',  'Blackened Industrial Metal',  
    'Pagan Black',  'Atmospheric Black Metal, Melodic Black Metal',  'Folk Black Metal'] 
T07=['Blackened Folk',  'Epic Black',  'Black Metal, Dark Ambient',  
    'Depressive Black Metal, Brutal Death Metal',  'Black Metal, Dungeon Synth',  'Atmospheric Depressive Black Metal',  
    'Depressive Black Metal, Cloud Rap',  'Symphonic Black Metal, Dark Ambient (early)',  'Blackened Grindcore',  
    'Various, Post-Black Metal',  'Blackened Goregrind',  'Black Metal, Ambient (early)'] 
T08=['Blackened Symphonic Pagan Metal',  'Post-Black Metal (early',  'Experimental Symphonic Black Metal',  
    'Progressive Post-Black Metal',  'Blackened Sludge Metal',  'Blackened Progressive Death',  
    'Folk Black',  'Industrial Black Metal (early)',  'Symphonic Blackened Death Metal (early)',  
    'Atmospheric Melodic Black Metal',  'Black Metal, Harsh Noise',  'Experimental Symphonic Black'] 
T09=['Technical Blackened Death Metal',  'Melodic Black Metal, Power Metal',  'Blackened Progressive Metal',  
    'Symphonic Black Metal with classical influences',  'Experimental Depressive Black Metal early)',  'Blackened Experimental Metal',  
    'Atmospheric Black Metal, Ambient',  'Melodic Black Metal with Folk influences',  'Gothic Black']  
T10=['Ambient, Black Metal',  'Epic Atmospheric Black Metal',  'Brutal Blackened Death Metal',  
    'Depressive Black Metal, Experimental, Ambient',  'Black Folk Metal',  'Hardcore Punk, Black Metal',  
    'Blackened Funeral Doom',  'Atmospheric Black Metal, Dark Ambient',  'Viking Black']
T11=['Blackened Melodic Death Metal',  'Blackened Epic Heavy Metal',  'Blackened Death Metal',  
    'Blackened Heavy Metal',  'Ambient Black Metal', 
    'Symphonic Black',  'Symphonic Blackened Death Metal',  'Melodic Black',  
    'Pagan Black Metal',  'Raw Black Metal',  'Black Metal (early)']
T12=['Atmospheric Post-Black Metal',  'Depressive Black Metal',  'Depressive Black',  'Blackened Stoner',  'Black Metal, Various',  
    'Melodic Epic Black Metal',  'Blackened Speed Metal',  'Blackened Thrash Metal',  
    'Blackened Heavy',  'Melodic Black Metal (early)',  'Black Metal with Punk',  
    'Experimental Black Metal, Various',  'Industrial Black Metal',  'Blackened Death Metal with Thrash elements',  
    'Blackened Doom',  'Experimental Ambient Black Metal',  'Symphonic Black Metal']
T13=['Experimental Black Metal', 'Post-Black Metal',  'Epic Black Metal',  'Blackened',  
    'Symphonic Progressive Black Metal',  'Atmospheric Pagan Black Metal (early)',  'Symphonic Black Metal (early)',  
    'Raw Black',  'Blackened Progressive Death Metal',  'Epic Symphonic Black Metal (early)',  
    'Symphonic Melodic Black',  'Atmospheric Black',  'Blackened Brutal Death Metal',
    'Depressive Black Metal (early)',  'Avant-garde Black Metal',  'Progressive Black']  
SongType=[T00,T01,T02,T03,T04,T05,T06,T07,T08,T09,T10,T11,T12,T13]